In [64]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import os
import json

In [65]:
model_version = '1.0'
# latent factor의 수
num_features = 30
# iteration 수
iterations = 140
# learning_rate 
learning_rate = 1e-1
# lambda
lambda_ = 1
# count_weight
count_weight = 20

In [66]:
# 저장된 모델 버전 및 체크포인트 경로 설정
checkpoint_path = f'../model/CF/train/{model_version}/'

# CSV 파일 불러오기
UI_temp = pd.read_csv(checkpoint_path + 'UI_temp.csv').drop('userId', axis=1)
UI_count_div = pd.read_csv(checkpoint_path + 'UI_count_div.csv').drop('userId', axis=1)
test_for_loss = pd.read_csv(checkpoint_path + 'test_for_loss.csv')
test_data_df = pd.read_csv(checkpoint_path + 'test_data_df.csv')
user_category_not_valid_df = pd.read_csv(checkpoint_path + 'user_category_not_valid.csv')
category_df = pd.read_csv(checkpoint_path + 'category.csv')

In [67]:
UI_test = UI_temp.copy()
# UI_test의 값을 모두 0으로 초기화
UI_test = UI_test.map(lambda x: 0.0)
for user in UI_temp.index:
    for item in UI_temp.columns:
        # test에 해당 user-item이 있는 경우 해당 user-item의 평균을 기록
        if item in test_data_df[test_data_df['userId'] == user]['옷 조합'].values:
            UI_test.loc[user, item] = test_data_df[(test_data_df['userId'] == user) & (test_data_df['옷 조합'] == item)]['평균기온(°C)'].mean()

In [68]:
bins = category_df.values[0, 1:]
labels = category_df.columns[1:]

In [69]:
# labels의 dtype을 float64로 변경
labels = labels.astype('float64')

In [70]:
# CF를 위한 초기값 설정
Y = np.array(UI_temp) 
Y = Y.T
count = np.array(UI_count_div)
count = count.T
print(Y.shape)
R = Y != 0 
n_u = Y.shape[1]
n_o = Y.shape[0]

(568, 14)


In [71]:
# 기록이 존재하는 값의 평균을 구함
o_sum = Y.sum(axis=1)
o_count = R.sum(axis=1)
o_mean = o_sum / o_count
o_mean = o_mean.reshape(-1, 1)

In [72]:
Y_test = np.array(UI_test)
Y_test = Y_test.T

In [73]:
Y_stand = Y - (o_mean * R)
Y_test_stand = Y_test - (o_mean * (Y_test != 0))

In [74]:
# user, outfit의 수
n_o, n_u = Y.shape

# (U,O)를 초기화하고 tf.Variable로 등록하여 추적
tf.random.set_seed(1234) # for consistent results
U = tf.Variable(tf.random.normal((n_u,  num_features),dtype=tf.float64),  name='U')
O = tf.Variable(tf.random.normal((n_o, num_features),dtype=tf.float64),  name='O')
b = tf.Variable(tf.random.normal((1,          n_u),   dtype=tf.float64),  name='b')

# optimizer 초기화
optimizer = keras.optimizers.Adam(learning_rate = learning_rate)

In [75]:
def load_variables_optimizer(variables, optimizer, checkpoint_path):
    checkpoint = tf.train.Checkpoint(variables=variables, optimizer=optimizer)
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path))

In [76]:
# 훈련된 변수 불러오기
load_variables_optimizer({"O": O, "U": U, "b": b}, optimizer,  checkpoint_path)

In [77]:
O

<tf.Variable 'O:0' shape=(568, 30) dtype=float64, numpy=
array([[ 0.00586259,  0.00298689, -0.00033087, ..., -0.00941248,
        -0.00415345,  0.00296635],
       [ 0.00587581,  0.00294877, -0.00033374, ..., -0.00927998,
        -0.00417229,  0.00293866],
       [-0.00435575, -0.00435865, -0.0056923 , ...,  0.00254545,
         0.00076383,  0.00728013],
       ...,
       [-0.00434739, -0.004443  , -0.0057135 , ...,  0.00251466,
         0.00084568,  0.00728021],
       [-0.00052874, -0.00143907, -0.0006285 , ..., -0.00082871,
         0.00037749,  0.0015894 ],
       [-0.00068142, -0.00444404,  0.00274962, ...,  0.00265676,
         0.00847348, -0.0005166 ]])>

In [78]:
def cofi_cost_func_v(O, U, b, Y, R, lambda_):
    j = (tf.linalg.matmul(O, tf.transpose(U)) + b - Y )*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(O**2) + tf.reduce_sum(U**2))
    return J

In [79]:
def test(O, U, b, o_mean, count, count_weight, df, UI_temp, labels, user_category_not_valid) :
    # 예측을 수행하기 위해 모든 user-item에 대한 예측값을 계산
    p = np.matmul(O.numpy(), np.transpose(U.numpy())) + b.numpy()
    # user_category_not_valid에 해당하지 않는 경우에 대해 precision, recall, f1_score 계산
    # 평균을 위한 초기화
    precision_m, recall_m, f1_score_m, count_m = 0, 0, 0, 0
    for i in range(UI_temp.shape[0]):
        for category in labels:
            
            # 실제 온도
            # 평균을 적용하고 temp를 빼서 값이 작을수록 실제 온도에 가깝도록 함. 이 때 각 user-item의 사용 횟수를 가중하여 많이 사용한 item이 추천되도록 함
            pm = np.power(p + o_mean - category, 2)  -count * count_weight
            my_predictions = pm[:,i]

            # sort predictions
            ix = tf.argsort(my_predictions, direction='ASCENDING')

            df_predict = UI_temp[UI_temp.columns[ix[0:3]]].copy()
            df_predict = df_predict.round(0)
            # df_predict의 columns와 test_data_df의 '옷 조합' column을 비교하여 일치하는 경우의 개수를 계산
            predict = df_predict.columns.astype(str)
            
            if i+1 in user_category_not_valid and category in user_category_not_valid[i+1]:
                continue
            
            label = df[(df['userId'] == i+1) & (df['평균기온(°C)'] == category)]['옷 조합'].astype(str)
            # label이 UI_temp의 column에 포함되지 않는다면 제외
            label = label[label.isin(UI_temp.columns)]
            # label에 어떠한 옷 조합도 포함되지 않을 시 지표를 측정하지 않음
            if label.shape[0] == 0:
                continue
            
            count_m += 1
            precision = len(set(predict) & set(label)) / len(set(predict))
            recall = len(set(predict) & set(label)) / len(set(label))
            if precision + recall == 0: 
                f1_score = 0
            else:
                f1_score = 2 * (precision * recall) / (precision + recall)
            precision_m += precision
            recall_m += recall
            f1_score_m += f1_score
    precision_m /= count_m
    recall_m /= count_m
    f1_score_m /= count_m
    return precision_m, recall_m, f1_score_m

In [80]:
precision, recall, f1_score = test(O, U, b, o_mean, count, count_weight, test_data_df, UI_temp, labels, user_category_not_valid_df)

In [81]:
print(f'loss: {cofi_cost_func_v(O, U, b, Y_test_stand, R, lambda_)}, 평균 precision: {precision}, 평균 recall: {recall}, 평균 f1_score: {f1_score}')

loss: 184.93375707303306, 평균 precision: 0.21739130434782608, 평균 recall: 0.11633885438233263, 평균 f1_score: 0.14745592571679528


In [88]:
data = {'loss': [cofi_cost_func_v(O, U, b, Y_test_stand, R, lambda_).numpy().astype('float64')], 'precision': [precision], 'recall': [recall], 'f1_score': [f1_score]}

In [89]:
json_data = json.dumps(data)
print(json_data)

{"loss": [184.93375707303306], "precision": [0.21739130434782608], "recall": [0.11633885438233263], "f1_score": [0.14745592571679528]}
